In [ ]:
import argparse
import pickle
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from utils.dataset_utils import get_dataset, get_data_from_dataloader, get_loading_parameters
from utils.nn_utils import get_rotated_mean, make_rotation_matrix, hitRate_generic, so2_rotate_subspaces
from utils.plotting_utils import plot_extra_dims, plot_images_distributions, \
    plot_mixture_neurreps, add_image_to_ax, add_distribution_to_ax_torus, save_embeddings_on_circle, yiq_embedding, \
    plot_embeddings_eval_torus, plot_projected_embeddings_pca, plot_std_distribution, plot_cylinder, \
    plot_image_mixture_rec_all, plot_clusters
from utils.plotting_utils_so3 import visualize_so3_probabilities
from utils.torch_utils import torch_data_to_numpy
from utils.disentanglement_metric import dlsbd_metric_mixture, dlsbd_metric_mixture_monte
from utils.model_utils import reload_model, get_embeddings, get_reconstructions, sigmoid, get_n_clusters_noise
from datasets.equiv_dset import EquivDataset
from dataset_generation.modelnet_efficient import ModelNetUniqueDataset, ModelNetDataset

In [ ]:
save_folder = "test1chamfer3"
model_name = "Dmodelnet_efficient-Dmodelnet10-Linfonce-ED20-N4-Mresnet-Aresnet-IDtest1chamfer3-S17"
autoencoder = "ae"
checkpoints_dir = "./saved_models"
neptune_user = "laprhanabi"
api_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmZWQ5MjRjMy0xMzI5LTQ5ZDAtYWEzYS04YjQ0YTk4NTgxYmYifQ=="

In [ ]:
os.listdir(os.path.join(os.path.dirname(os.getcwd()),"saved_models"))

In [ ]:
model_dir = os.path.join(os.path.dirname(os.getcwd()),"saved_models", "modelnet_efficient", save_folder,model_name)
meta_file = os.path.join(model_dir, 'metadata.pkl')
args = pickle.load(open(meta_file, 'rb'))['args']
print("Arguments", args)
save_folder = os.path.join(".", "visualizations",model_name)
os.makedirs(save_folder, exist_ok=True)
if args.gpu != "":```
    device = "cpu"
else:
    device = torch.device("cuda:" + args.gpu if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
device

In [ ]:
model, decoder = reload_model(model_dir, autoencoder, device=device)
model_path = os.path.join(checkpoints_dir, model_name)

In [ ]:
neptune_id_file = os.path.join(model_dir, 'neptune.txt')

if neptune_user != "":
    from utils.neptune_utils import reload_neptune_run

    run = reload_neptune_run(neptune_user, "non-free", neptune_id_file, api_token=api_token)
else:
    run = None

In [ ]:
# Define a pytorch Bayesian model for classifying encoded data
from models.classifiers import SimpleClassifierDropout
classif = SimpleClassifierDropout(args.extra_dim, hidden_layers_list=[20, 15, 10], num_classes = 10)

In [ ]:

classif